In [2]:
import pandas as pd


In [4]:
df=pd.read_excel("Data/British Airways Summer Schedule Dataset - Forage Data Science Task 1 - Copy.xlsx",sheet_name=0)

In [6]:
df.columns

Index(['FLIGHT_DATE', 'FLIGHT_TIME', 'TIME_OF_DAY', 'AIRLINE_CD', 'FLIGHT_NO',
       'DEPARTURE_STATION_CD', 'ARRIVAL_STATION_CD', 'ARRIVAL_COUNTRY',
       'ARRIVAL_REGION', 'HAUL', 'AIRCRAFT_TYPE', 'FIRST_CLASS_SEATS',
       'BUSINESS_CLASS_SEATS', 'ECONOMY_SEATS', 'TIER1_ELIGIBLE_PAX',
       'TIER2_ELIGIBLE_PAX', 'TIER3_ELIGIBLE_PAX'],
      dtype='object')

In [7]:
df.head(5)

,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,FLIGHT_NO,DEPARTURE_STATION_CD,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,2025-09-02,14:19:00,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,2025-06-10,06:42:00,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,2025-10-27,15:33:00,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40
3,2025-06-15,18:29:00,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,A320,0,8,172,0,16,54
4,2025-08-25,20:35:00,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,A320,0,13,167,0,6,27


In [9]:
group_cols = ['HAUL', 'ARRIVAL_REGION', 'TIME_OF_DAY']

In [22]:
assumptions = {
    'SHORT':  {'Tier1': 0.002, 'Tier2': 0.015, 'Tier3': 0.08},
    'LONG':   {'Tier1': 0.02,  'Tier2': 0.08,  'Tier3': 0.25}
}
print(df['HAUL'].unique())

['LONG' 'SHORT']


In [24]:
df['TOTAL_PAX'] = df['FIRST_CLASS_SEATS'] + df['BUSINESS_CLASS_SEATS'] + df['ECONOMY_SEATS']



In [25]:
def get_eligible(row, tier):
    haul_type = row['HAUL']
    base = assumptions.get(haul_type, {'Tier1':0, 'Tier2':0, 'Tier3':0})
    return row['TOTAL_PAX'] * base[tier]

df['TIER1_ELIGIBLE_PAX'] = df.apply(lambda r: get_eligible(r, 'Tier1'), axis=1)
df['TIER2_ELIGIBLE_PAX'] = df.apply(lambda r: get_eligible(r, 'Tier2'), axis=1)
df['TIER3_ELIGIBLE_PAX'] = df.apply(lambda r: get_eligible(r, 'Tier3'), axis=1)

In [26]:
lookup = df.groupby(group_cols)[['TIER1_ELIGIBLE_PAX', 'TIER2_ELIGIBLE_PAX', 'TIER3_ELIGIBLE_PAX']].mean().reset_index()


In [27]:
print(lookup.head(10))


   HAUL ARRIVAL_REGION TIME_OF_DAY  TIER1_ELIGIBLE_PAX  TIER2_ELIGIBLE_PAX  \
0  LONG           Asia   Afternoon            5.838553           23.354211   
1  LONG           Asia     Evening            5.791068           23.164272   
2  LONG           Asia   Lunchtime            5.780267           23.121067   
3  LONG           Asia     Morning            5.825854           23.303415   
4  LONG    Middle East   Afternoon            5.923522           23.694088   
5  LONG    Middle East     Evening            5.846368           23.385473   
6  LONG    Middle East   Lunchtime            5.693671           22.774684   
7  LONG    Middle East     Morning            5.807952           23.231807   
8  LONG  North America   Afternoon            5.836656           23.346624   
9  LONG  North America     Evening            5.843269           23.373078   

   TIER3_ELIGIBLE_PAX  
0           72.981908  
1           72.388350  
2           72.253333  
3           72.823171  
4           74.044025

In [31]:
lookup.to_csv("lookup",index=False)